In [19]:
__file__ = '/Users/hamel/github/docs-explore/custom/scratchpad.ipynb'

In [17]:
from nbconvert import MarkdownExporter
from nbconvert.preprocessors import TagRemovePreprocessor
import traitlets
from pathlib import Path

def exporter(cls=MarkdownExporter, template_file=None):
    c = traitlets.config.get_config()
    c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell",)
    c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
    c.TagRemovePreprocessor.remove_input_tags = ('remove_input',)
    c.TagRemovePreprocessor.enabled = True
    exporter = cls(c)
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.anchor_link_text = ' '
    exporter.template_file = 'ob.tpl'
    exporter.template_paths.append(str(Path(__file__).parent/'templates'))
    return exporter

In [18]:
exp = exporter()
print(exp.from_filename('Custom_Doc_System.ipynb')[0])

---
title: my hello page title
description: my hello page description
hide_table_of_contents: true
---
```python
# from IPython.core.magic import register_cell_magic
# import tempfile, os

# @register_cell_magic
# def mf(line, cell):
#     with tempfile.NamedTemporaryFile() as fp:
#         cell_code = [s.encode('utf-8') for s in cell]
#         preamble = [b'from metaflow import *'] 
#         fp.writelines(cell_code)
#         fp.seek(0)
#         real_command = f"python {fp.name} {line}"
#         disp_command = f"python flow.py {line}"
#         print(f">>> {disp_command}")
#         os.system(real_command)
```

```python
! echo hello
```

    hello


```python
%%writefile myflow.py

from metaflow import FlowSpec, step

class MyFlow(FlowSpec):
    
    @step
    def start(self):
        print('this is the start')
        self.next(self.end)
    
    @step
    def end(self):
        print('this is the end')

if __name__ == '__main__':
    MyFlow()
```

    Overwriting myflow.py


``